In [11]:
import pandas as pd

# Path to the dataset
data_path = "/Users/alexo/Desktop/VIP/data/data_sony_clean.csv"

# Read just the first row to get columns
df_header = pd.read_csv(data_path, nrows=1)
columns = df_header.columns.tolist()

print("Existing columns in the dataset:")
print(columns)

Existing columns in the dataset:
['timestamp', 'data_timestamp', 'date', 'buildingid', 'floorid', 'resourceid', 'resourcetype', 'temperature_value', 'temperature_rating', 'light_value', 'light_rating', 'co2_value', 'co2_rating', 'humidity_value', 'humidity_rating', 'radon_value', 'radon_rating', 'airquality_value', 'airquality_rating', 'type', 'bookable', 'size']


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Path to the dataset
data_path = "/Users/alexo/Desktop/VIP/data/data_sony_clean.csv"

# Load a sample (first 100,000 rows) for exploration to handle large size
df_sample = pd.read_csv(
    data_path, nrows=100000, parse_dates=["timestamp"]
)  # Adjust 'timestamp' if named differently

# Basic info
print("Sample shape:", df_sample.shape)
print("\nData types:\n", df_sample.dtypes)
print("\nMissing values per column:\n", df_sample.isnull().sum())
print("\nSummary statistics:\n", df_sample.describe())

# Check for duplicates
duplicates = df_sample.duplicated().sum()
print(f"\nNumber of duplicate rows in sample: {duplicates}")

Sample shape: (100000, 22)

Data types:
 timestamp              object
data_timestamp         object
date                   object
buildingid              int64
floorid                 int64
resourceid             object
resourcetype           object
temperature_value     float64
temperature_rating     object
light_value           float64
light_rating           object
co2_value             float64
co2_rating             object
humidity_value        float64
humidity_rating        object
radon_value           float64
radon_rating           object
airquality_value      float64
airquality_rating      object
type                   object
bookable                 bool
size                  float64
dtype: object

Missing values per column:
 timestamp             0
data_timestamp        0
date                  0
buildingid            0
floorid               0
resourceid            0
resourcetype          0
temperature_value     0
temperature_rating    0
light_value           0
light_rating    

In [17]:
#!/usr/bin/env python
"""
Create VAE-ready dataset from pre-processed Sony Nimway data.
Input : /Users/alexo/Desktop/VIP/data/data_sony_clean.csv
Output: ../../data/vae_data.csv
"""

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings("ignore")

# -------------------------------
# Configuration
# -------------------------------
INPUT_PATH = Path("/Users/alexo/Desktop/VIP/data/data_sony_clean.csv")
OUTPUT_DIR = Path("../../data")
OUTPUT_PATH = OUTPUT_DIR / "vae_data.csv"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

SENSOR_COLS = [
    "temperature_value",
    "humidity_value",
    "co2_value",
    "light_value",
    "radon_value",
    "airquality_value",
]
CONTEXT_COLS = ["timestamp", "resourceid"]
FINAL_COLS = CONTEXT_COLS + SENSOR_COLS

# -------------------------------
print("Loading dataset...")
df = pd.read_csv(INPUT_PATH, parse_dates=["timestamp"])  # <-- ISO-8601 works here

print(f"Original shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# -------------------------------
# 1. Feature Selection
# -------------------------------
print("\nSelecting features...")
missing = [c for c in FINAL_COLS if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")
df = df[FINAL_COLS].copy()

# -------------------------------
# 2. Data Cleaning & Sorting
# -------------------------------
print("Cleaning and sorting...")
df = (
    df.drop_duplicates().sort_values(["resourceid", "timestamp"]).reset_index(drop=True)
)

# ---- FIXED TIMESTAMP PARSING (only needed if read_csv did not parse it) ----
df["timestamp"] = pd.to_datetime(
    df["timestamp"], utc=True, errors="coerce", infer_datetime_format=True
)
n_bad = df["timestamp"].isna().sum()
if n_bad:
    print(f"Warning: Dropping {n_bad} rows with unparsable timestamps")
    df = df.dropna(subset=["timestamp"])

# -------------------------------
# 3. Per-resource Normalization (MinMax [0,1])
# -------------------------------
print("Normalizing sensor values per resource...")
scaler = MinMaxScaler()


def normalize_group(g):
    sensors = g[SENSOR_COLS]
    if sensors.isnull().all().all():
        return g
    scaled = scaler.fit_transform(sensors)
    g = g.copy()
    g[SENSOR_COLS] = pd.DataFrame(scaled, columns=SENSOR_COLS, index=g.index)
    return g


df_norm = df.groupby("resourceid", group_keys=False).apply(normalize_group)

# -------------------------------
# 4. Final clean-up
# -------------------------------
df_norm[SENSOR_COLS] = df_norm[SENSOR_COLS].ffill().bfill()
df_final = df_norm.dropna(subset=SENSOR_COLS, how="any")

# -------------------------------
# 5. Save
# -------------------------------
print(f"Saving to {OUTPUT_PATH}...")
df_final.to_csv(OUTPUT_PATH, index=False)

print("\nVAE dataset ready!")
print(f"   Shape : {df_final.shape}")
print(f"   File  : {OUTPUT_PATH.resolve()}")
print(df_final.head())

Loading dataset...
Original shape: (702169, 22)
Columns: ['timestamp', 'data_timestamp', 'date', 'buildingid', 'floorid', 'resourceid', 'resourcetype', 'temperature_value', 'temperature_rating', 'light_value', 'light_rating', 'co2_value', 'co2_rating', 'humidity_value', 'humidity_rating', 'radon_value', 'radon_rating', 'airquality_value', 'airquality_rating', 'type', 'bookable', 'size']

Selecting features...
Cleaning and sorting...
Normalizing sensor values per resource...
Saving to ../../data/vae_data.csv...

VAE dataset ready!
   Shape : (701115, 8)
   File  : /Users/alexo/Desktop/VIP/github-repo/VIP/data/vae_data.csv
                         timestamp                            resourceid  \
0 2024-02-11 23:59:41.269000+00:00  12834a49-ce1d-4f43-949a-996dbb088f74   
1 2024-02-11 23:59:41.323000+00:00  12834a49-ce1d-4f43-949a-996dbb088f74   
2 2024-02-12 00:04:44.262000+00:00  12834a49-ce1d-4f43-949a-996dbb088f74   
3 2024-02-12 00:04:44.336000+00:00  12834a49-ce1d-4f43-949a-996dbb0

In [18]:
# -------------------------------------------------
#  Remove duplicates from vae_data.csv (in-place)
# -------------------------------------------------
import pandas as pd
from pathlib import Path

# ---- 1. Path (adjust only if you move the notebook) ----
CSV_PATH = Path("/Users/alexo/Desktop/VIP/github-repo/VIP/data/vae_data.csv")

# ---- 2. Load ------------------------------------------------
print(f"Loading {CSV_PATH.name} ...")
df = pd.read_csv(CSV_PATH, parse_dates=["timestamp"])
print(f"   rows before: {len(df):,}")

# ---- 3. Drop duplicates ------------------------------------
#   * keep='first' → retain the first occurrence
#   * subset=None  → compare **all** columns
df_clean = df.drop_duplicates(keep="first").reset_index(drop=True)

# ---- 4. Summary ---------------------------------------------
dup_count = len(df) - len(df_clean)
print(f"   duplicates removed: {dup_count:,}")
print(f"   rows after : {len(df_clean):,}")

# ---- 5. Overwrite the original file -------------------------
df_clean.to_csv(CSV_PATH, index=False)
print(f"\nFile overwritten → {CSV_PATH}")
print("Done!")

Loading vae_data.csv ...
   rows before: 701,115
   duplicates removed: 0
   rows after : 701,115

File overwritten → /Users/alexo/Desktop/VIP/github-repo/VIP/data/vae_data.csv
Done!


In [23]:
# =====================================================
#  PREPROCESSING FOR ADAPTIVE VAE (ONLINE + HUMAN-IN-LOOP)
#  Input : vae_data.csv
#  Output: vae_final_streaming.csv + scaler_per_room.pkl
# =====================================================
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings("ignore")

# -------------------------------
# 1. Load Cleaned VAE Data
# -------------------------------
CSV_PATH = Path("/Users/alexo/Desktop/VIP/github-repo/VIP/data/vae_data.csv")
print(f"Loading {CSV_PATH.name}...")
df = pd.read_csv(CSV_PATH, parse_dates=["timestamp"])
print(f"   Rows: {len(df):,}")

# -------------------------------
# 2. Sanity Checks
# -------------------------------
assert df.duplicated().sum() == 0, "Duplicates still present!"
assert df["timestamp"].isna().sum() == 0, "Missing timestamps!"

SENSOR_COLS = [
    "temperature_value",
    "humidity_value",
    "co2_value",
    "light_value",
    "radon_value",
    "airquality_value",
]

# -------------------------------
# 3. Sort & Time Gaps
# -------------------------------
print("Sorting and checking time gaps per room...")
df = df.sort_values(["resourceid", "timestamp"]).reset_index(drop=True)
df["time_diff_min"] = (
    df.groupby("resourceid")["timestamp"].diff().dt.total_seconds() / 60
)
irregular = df[df["time_diff_min"] > 30]
print(
    f"   Irregular gaps (>30 min): {len(irregular):,} rows in {irregular['resourceid'].nunique()} rooms"
)

Loading vae_data.csv...
   Rows: 701,115
Sorting and checking time gaps per room...
   Irregular gaps (>30 min): 169 rows in 9 rooms


In [24]:
# -------------------------------
# 4. Per-Room MinMax Scaling + Save Scalers
# -------------------------------
print("Scaling sensors per resourceid...")
scalers = {}


def scale_group(group):
    rid = group["resourceid"].iloc[0]
    scaler = MinMaxScaler()
    sensors = group[SENSOR_COLS].copy()
    scaled = scaler.fit_transform(sensors)
    group[SENSOR_COLS] = pd.DataFrame(scaled, columns=SENSOR_COLS, index=group.index)
    scalers[rid] = scaler
    return group


df_scaled = df.groupby("resourceid", group_keys=False).apply(scale_group)
print(f"   Scalers saved for {len(scalers)} rooms")

Scaling sensors per resourceid...
   Scalers saved for 9 rooms


In [25]:
# -------------------------------
# 5. Interpolate Small Gaps (Per Room)
# -------------------------------
print("Interpolating small gaps per room...")
df_interp = df_scaled.copy()

# Only interpolate within each room
for rid, group in df_interp.groupby("resourceid"):
    df_interp.loc[group.index, SENSOR_COLS] = group[SENSOR_COLS].interpolate(
        method="linear", limit_direction="both"
    )

# Final NaN check
assert df_interp[SENSOR_COLS].isna().sum().sum() == 0, "NaNs remain!"
print("   Interpolation complete.")

Interpolating small gaps per room...
   Interpolation complete.


In [26]:
# -------------------------------
# 6. Create Sliding Window Sequences (VAE Input)
# -------------------------------
WINDOW_SIZE = 10  # 10 timesteps (~50 min if 5-min data)
print(f"Creating sliding windows (size={WINDOW_SIZE})...")

sequences = []


def create_sequences(group):
    if len(group) < WINDOW_SIZE:
        return
    data = group[SENSOR_COLS].values
    timestamps = group["timestamp"].values
    resource_id = group["resourceid"].iloc[0]

    for i in range(len(data) - WINDOW_SIZE + 1):
        seq = data[i : i + WINDOW_SIZE]
        row = {
            "seq_id": f"{resource_id}_{i}",
            "resourceid": resource_id,
            "start_time": pd.Timestamp(timestamps[i]),
        }
        # Flatten sequence: t0_feature1, t0_feature2, ..., t9_feature6
        for t in range(WINDOW_SIZE):
            for f_idx, feature in enumerate(SENSOR_COLS):
                row[f"{feature}_t{t}"] = seq[t, f_idx]
        sequences.append(row)


# Apply per room
_ = df_interp.groupby("resourceid", group_keys=False).apply(create_sequences)

df_seq = pd.DataFrame(sequences)
print(f"   Sequences created: {len(df_seq):,}")
print(f"   Features per sequence: {len(df_seq.columns)}")

Creating sliding windows (size=10)...
   Sequences created: 701,034
   Features per sequence: 63


In [27]:
# -------------------------------
# 7. Save Final Files
# -------------------------------
OUTPUT_DIR = CSV_PATH.parent
FINAL_CSV = OUTPUT_DIR / "vae_final_streaming.csv"
SCALER_PKL = OUTPUT_DIR / "scaler_per_room.pkl"

df_seq.to_csv(FINAL_CSV, index=False)
with open(SCALER_PKL, "wb") as f:
    pickle.dump(scalers, f)

print(f"\nPREPROCESSING COMPLETE!")
print(f"   → Sequences: {FINAL_CSV}")
print(f"   → Scalers:   {SCALER_PKL}")
print(f"   → Total sequences: {len(df_seq):,}")


PREPROCESSING COMPLETE!
   → Sequences: /Users/alexo/Desktop/VIP/github-repo/VIP/data/vae_final_streaming.csv
   → Scalers:   /Users/alexo/Desktop/VIP/github-repo/VIP/data/scaler_per_room.pkl
   → Total sequences: 701,034
